In [1]:
import requests
import re
from bs4 import BeautifulSoup as bs4
import pandas as pd
import numpy as np
pd.set_option('display.max_rows', None)

In [2]:
def ScrapeSongsOfYear(YearSongsURL):
    page = requests.get(YearSongsURL)
    soup = bs4(page.content, 'html.parser')
    results = soup.find(id="main-wrapper")
    table = results.find('tbody')
    rows = table.find_all('tr')

    return rows

In [3]:
def ExtractData(data):
    df = pd.DataFrame(columns=["Index","Song","Artist","Genre1"])#,"Genre2"])
    songlinks = []
    
    for row in data:
        cols = row.find_all('td')
        link_elem = cols[1].find('a')
        if link_elem is not None: 
            link = link_elem.get('href')
            songlinks.append(link)

            df = df.append({
                "Index": cols[0].text.strip(),
                "Song": cols[1].text.strip(),
                "Artist": cols[2].text.strip(),
            }, ignore_index=True)
        
    return df,songlinks

In [7]:
def getSongGenre(links, df, start_index=0):
    for link in links:
        page = requests.get(link)
        soup = bs4(page.content, 'html.parser')
        table = soup.find('table', class_='infobox').find('th', string='Genre') 
        if table is None:
            df.loc[start_index, "Genre1"] = "None"
#             df.loc[start_index, "Genre2"] = "None"
        else:
            genres = table.find_parent('tr').find('td')
            genre_names = [genre.strip() for genre in genres.text.split('\n') if genre.strip()]
            superscript_pattern = r'\[\d+\]'
            genre_names = [re.sub(superscript_pattern, '', text) for text in genre_names]
            df.loc[start_index, "Genre1"] = genre_names[0]
#             If i want to get top two genres, but too many None
#             if len(genre_names) > 1:
#                 df.loc[start_index, "Genre2"] = genre_names[1]
#             else:
#                 df.loc[start_index, "Genre2"] = "None"
        start_index += 1
    return df
# links = ['https://en.wikipedia.org/wiki/Heat_Waves', 
#          'https://en.wikipedia.org/wiki/As_It_Was']
getSongGenre(links,df)

,Index,Song,Artist,Genre1
0,1,“Heat Waves“,Glass Animals,Psychedelic pop
1,2,“As It Was“,Harry Styles,Synth-pop
2,3,“Stay“,The Kid Laroi and Justin Bieber,Pop rap
3,4,“Easy on Me“,Adele,Pop
4,5,“Shivers“,Ed Sheeran,Dance-pop
5,6,“First Class“,Jack Harlow,None
6,7,“Big Energy“,Latto,Funk-poppop rap
7,8,“Ghost“,Justin Bieber,Pop rock
8,9,“Super Gremlin“,Kodak Black,None
9,10,“Cold Heart (Pnau remix)“,Elton John and Dua Lipa,Dance


In [66]:
def getArtistGender(links):
    genders =[]
    for link in links:
        page = requests.get(link)
        soup = bs4(page.content, 'html.parser')
        section = soup.find(class_='mw-body-content mw-content-ltr')
        intro = section.find_all('p')[1].text
        if 'band' in intro:
            genders.append('Band')
        elif 'collaboration' in intro or 'ensemble' in intro:
            genders.append('Collab')
        elif 'she' in intro or 'her' in intro:
            genders.append('Female')
        elif 'he' in intro or 'his' in intro:
            genders.append('Male')
        else:
            genders.append('Unknown')
    return genders

genders = getArtistGender(links)

In [67]:
df['Genders'] = genders #fix morgan wallen band entry
df

,Index,Song,Artist,Genre1,Genders
0,1,“Heat Waves“,Glass Animals,NaN,Band
1,2,“As It Was“,Harry Styles,NaN,Male
2,3,“Stay“,The Kid Laroi and Justin Bieber,NaN,Collab
3,4,“Easy on Me“,Adele,NaN,Female
4,5,“Shivers“,Ed Sheeran,NaN,Male
5,6,“First Class“,Jack Harlow,NaN,Male
6,7,“Big Energy“,Latto,NaN,Female
7,8,“Ghost“,Justin Bieber,NaN,Male
8,9,“Super Gremlin“,Kodak Black,NaN,Male
9,10,“Cold Heart (Pnau remix)“,Elton John and Dua Lipa,NaN,Collab


In [54]:
df,links = ExtractData(data)

['https://en.wikipedia.org/wiki/Heat_Waves', 'https://en.wikipedia.org/wiki/As_It_Was', 'https://en.wikipedia.org/wiki/Stay_(The_Kid_Laroi_and_Justin_Bieber_song)', 'https://en.wikipedia.org/wiki/Easy_on_Me', 'https://en.wikipedia.org/wiki/Shivers_(Ed_Sheeran_song)', 'https://en.wikipedia.org/wiki/First_Class_(song)', 'https://en.wikipedia.org/wiki/Big_Energy', 'https://en.wikipedia.org/wiki/Ghost_(Justin_Bieber_song)', 'https://en.wikipedia.org/wiki/Super_Gremlin', 'https://en.wikipedia.org/wiki/Cold_Heart_(Pnau_remix)', 'https://en.wikipedia.org/wiki/Wait_for_U', 'https://en.wikipedia.org/wiki/About_Damn_Time', 'https://en.wikipedia.org/wiki/Bad_Habits_(Ed_Sheeran_song)', 'https://en.wikipedia.org/wiki/Thats_What_I_Want', 'https://en.wikipedia.org/wiki/Enemy_(Imagine_Dragons_and_JID_song)', 'https://en.wikipedia.org/wiki/Industry_Baby', 'https://en.wikipedia.org/wiki/ABCDEFU', 'https://en.wikipedia.org/wiki/Need_to_Know_(Doja_Cat_song)', 'https://en.wikipedia.org/wiki/Wasted_on_You_(

In [5]:
Y2022 = "http://billboardtop100of.com/2022-2/"
Y2012 = "http://billboardtop100of.com/2012-2/"
Y2002 = "http://billboardtop100of.com/2002-2/"
Y1992 = "http://billboardtop100of.com/1992-2/"
Y1982 = "http://billboardtop100of.com/1982-2/"
Y1972 = "http://billboardtop100of.com/1972-2/"
Y1962 = "http://billboardtop100of.com/1962-2/"
Y1952 = "http://billboardtop100of.com/1952-2/"
# Different format
Y1942 = "http://billboardtop100of.com/1942-2/"

data = ScrapeSongsOfYear(Y2022)